<a href="https://colab.research.google.com/github/cyrus2281/notes/blob/main/MachineLearning/Recommenders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommenders

## Mathematical Recommenders

These examples are non-personalized recommendations


### Lift

$$
\text{Lift} = \frac{p(A,B)}{p(A)p(B)} = \frac{p(A|B)}{p(A)} = \frac{p(B|A)}{p(B)}
$$

- Symmetric
- If A and B are independent, then $p(A|B) = p(A)$
  - $p(A|B) / p(A) = 1$
- if increasing the probability of B increases the probability of A, then Lift > 1

### Hacker News Formula

Balancing Popularirty with Age

$$
\frac{f(\text{popularity})}{g(\text{age})} \\[1cm]
$$

$$
\text{score} = \frac{(\text{ups} - \text{downs} -1 )^{0.8}}{(\text{age}+2)^{\text{gravity}}} \times \text{penalty}
$$

- gravity = 1.8
- penalty = multiplier to implement "business rules" (e.g. penalize self-posts, "controversial" posts, + many more rules)

\

age starts from 2, to prevent division by zero

exponent of numerator is bigger than the exponent of the denominator, meaning denominator grows faster.

Age always overtakes popularity

\

exponent 0.8 causes sublinear growth.
meaning 0 → 100 worth more than 1000 → 1100



### Reddit Formula

$$
\text{score} = \text{sign}(\text{ups}-\text{down}) \times \log \{ \max(1, |\text{ups} - \text{downs}|) \} + \frac{\text{age}}{45000}
$$

- log of the absoulte value of net votes - sublinear curve - initial votes matter more - max since log 0 is not possible

\

- can be positive or negative
  - The more downvotes you get, the futher your score goes down

\

- age is in seconds from inception of reddit
- age is always positive
- newer links → more score
- reddit scores will forever increase linearly


### Google Page Rank

Logic: The page rank of a page is the probability I would end up on that page if I surfed the internet randomly for an infinite amount of time



#### Markov Models

- Simplest way to think about Markov Models are bigrams from NLP
- Build a probablistic language model
- Can ask "what is the probability of the next word in the sentence 'love' give the previous word was 'I'?" i.e., p(love | I )

**Bigrams**

- It's a bigram because we only consider 2 words at a time

We don't have to think of each item as a word, just a generic state: $x(t)$

"Markov" means $x(t)$ doesn't depend on any values 2 or more steps behind, only the immediate last value.
$$
p(x_t | x_{t-1}, x_{t-2}, \cdots, x_1) = p(x_t | x_{t-1})
$$



#### Transition Probability Matrix

- $A(i,j)$ tells use the probability of going to state j from state i
$$
A(i,j) = p(x_t = j | x_{t-1} = i )
$$

- Key: rows must sum to 1
    - Since it's a probability this must be true
    - If true, A is called a "stochastic matrix" or "Markov Matrix"
$$
\sum^M_{j=1}A(i,j) = \sum^M_{j=1}p(x_t=j|x_{t-1}=i) =1
$$


\

Example:

Weather is
- state 1 = Sunny
- state 2 = Rainy

Suppose:
- p( sunny | sunny ) = 0.9
- p( sunny | rainy ) = 0.1
- p( rainy | sunny ) = 0.1
- p( rainy | rainy ) = 0.9



#### Calculating Probabilities

$$
p(\text{rainy} | \text{sunny}) = \frac{\text{count}(\text{sunny} \rightarrow \text{rainy})}{\text{count}(\text{sunny})}
$$

Generalized

$$
p(B|A) = \frac{\text{count}(A \rightarrow B)}{\text{count}(A)}
$$

Now probability of a sentence would be

$$
p(x_1, \cdots, x_T) = p(x_1) \prod^T_{t=2} p(x_t | x_{t-1})
$$

Problem: If a bigram didn't apear in the train set, the probility would be 0 and anything × 0 is 0.

**Add-1 Smoothing**

$$
p(x_t=j|x_{t-1}=i) = \frac{\text{count}(i \rightarrow j)+\epsilon}{\text{count}(i)+\epsilon V} \\
$$

- Add a "fake count" to every possible bigram
  - ϵ can be any value, for example 1.
- V = Vocabulary size = number of unique words in dataset
- In this case, V=M (number of states) since each state is a word


e.g. p(and | and) never occurs but would get positve probability


#### Beta Posterior Mean

In our case, the equation is the beta posterior mean instead of only 2 possible outcomes, V possible outcomts

$$
E(\pi) = \frac{\alpha'}{\alpha'+\beta'} = \frac{\alpha+(\sum_{i=1}^N X_i)}{\alpha + \beta + N} \\
$$

#### State Distribution

$\pi_t$ = state probability distribution at time t

$\pi(t)$ is a row vector by convention

For the weather example,

$$
\pi_t = [p(x_t = \text{sunny}), p(x_t = \text{rainy})]
$$

**Future State Distribution**

Calculating the $\pi(t+1)$ use Bayes rule

$$
p(x_{t+1} = j) = \sum^M_{i=1} p(x_{t+1} = j, x_t =i) \\
= \sum^M_{i=1} p(x_{t+1}=j | x_t = i) p(x_t = i) \\
= \sum^M_{i=1} A(i,j)\pi(i) \\
= \pi_{t+1}(j)
$$

Since A is a matrix and $\pi(t)$ is a vector, we can express it in terms of matrix math
$$
\pi_{x+1}(j) = \sum^M_{i=1} A(i,j)\pi_t(i) \\
\pi_{t+1} = \pi_t A
$$

Further future

$$
\pi_{t+2} = \pi_t A^2 \\
\pi_{t+k} = \pi_t A^k \\
$$

For infinity

$$
\pi_\infty = \lim_{t\rightarrow \infty} \pi_0 A^t \\
\pi_\infty = \pi_\infty A
$$

This is just the eigenvalue problem
  - Give matrix A, find a vector and a scalar s.t. multiplying the vector by A is equivalent to stretching it be the scalar.

#### PageRank

Every page on the internet is a state in a Markov Model

The transition probablity is distributed equally amongst all links on a page
- p(dlc.com | lp.me ) = 0.5
- p(yt.com | lp.me ) = 0.5

In general, we can write the transition probability as:

$$
p(x_t =j | x_{x-1} = i) = \frac{1}{n(i)}
$$
if $i$ links to $j$, $n(i) = $ number of links on page $i$, otherwise $0$.

**Smoothing**

$$
G = 0.85A + 0.15U \\
U(i,j) = \frac{1}{M} \\
\forall i,j = 1 \dots M
$$

Find the limiting distribution of G - yields a vector of length M - these probabilities are the respective PageRanks for ech page on the internet

$$
\pi_\infty = \pi_\infty G
$$


**Perron-Frobenius Theorem**:
> If G is a valid Markov matrix and all its elements are positive then the stationary distribution and limiting distribution are the same
- Limiting Distribution: state distribution you'd arrive at after transitioning by G an infinite number of times
- Stationary Distribution: a state distribution that does not change after transitioning by G

## Statistics

### Smoothing (Dampening)

To resolve the issue of 0 in sample data when getting mean

$$
r = \frac{\sum^N_{i=1} X_i + \lambda \mu_0}{N+\lambda}
$$


- $\lambda$ some random small non-zero number
- $\mu_0$ the global avergage or just some middle value

\

For example:
- 1000 reviews of 4 star - μ = 3 - λ = 1 → 3.999
- 5 reviews of 4 star - μ = 3 - λ = 1 → 3.83
- 1 review of 4 star - μ = 3 - λ = 1 → 3.5

### Explore-Exploit Dilemma

Example 1

Imagine we want to find the slot machine with the highest win rate among 10 slot machines.

Traditional statistical test can tell us whether or not there's a significant difference between win rates between machines.

If playing each machine 100 times, meaning 1000 turns total, 900 (9/10) turns yielded a suboptimal reward.

Hence the dilemma, Play more or play less!


---

Example 2

Watching a bunach of YouTube videos on how to make eggs.

Now your reccomendations are filled with videos about making eggs

porbably suboptimal - once I've figured out how to make eggs, I don't want to watch more egg videos.

YouTube is not exploiting the fact that I watched eggs video and not exploring other topics

Should there be a stronger exploration component?

Maybe I'd like to seE movie trailers or machine learning videos

---

How do we strike a balance between these 2 opposing forces?

Smoothed average gives us one part of the solution

Making good things look worse and bad things look better


### Bayesian Method

Bayesian method automatically balances need to explore and exploit

- 2 fat distributions: explore both (totally random ranking)
- 2 skinny distributions: exploit both (nearly deterministic ranking)
- Mixed: explore and exploit co-exit


Completely automatic - does not require A/B testing



## Collaborative Filtering

Non-specific to any particular user, Score each item from 1 to M a number.

Basic algorithm is to make s(j) the average rating for j

$$
s(j) = \frac{\sum_{i \in \Omega_j} r_{ij}}{|\Omega_j|} \\
$$

- $\Omega_j$ = set of all users who rated item j
- $r_{ij}$ = rating user i gave item j

Translates to, average rating for a product is the sum of rating divided by the number of the ratings.

**Personalize the score**

s(i,j) can depend both on user i and item j

$$
s(i,j) = \frac{\sum_{i' \in \Omega_j} r_{i'j}}{|\Omega_j|} \\
$$

i' is just an index

i = 1 … N, N = number of users

j = 1 … M, M = number of items

$R_{N\times M}$ = user -item ratings matrix of size N × M


\

- User-item matrix is reminiscent of term-document.
- X(t,d) = # of time term t appears in document d
- In terms of recommender systems, can think of X(t,d) as "how much does t like the item d"




### Sparsity

One characteristic of the user items matrix that makes it unique to recommender systems is that it's Sparse.

- Term-document matrix is sparse because most entries are 0
- User-item matrix is sparse because most entries are **empty**

The average user does not interact with all items.


**Goal of Collaborative Filtering**

- Most of r(i,j) doesn't exist - this is good.

If every user has seen every item, then there's nothing to recommend

Goal:
> We want to guess what you might rate an item you haven't seen yet

$$
s(i,j) = \hat r (i,j) = \text{ guess what user i might rate item j}
$$

E.g. if we think you might rate some move a 5, we definitely want you to watch that movie.


### Regression

Since this is a regression probelm, the evaluation metric is going to be the mean squared error.

Outline:
- user-user collaborative filtering
- item-item collaborative filtering

$$
\text{MSE} = \frac{1}{|\Omega|} \sum_{i,j \in \Omega}(r_{ij} - \hat r_{ij})^2
$$

Ω = set of pairs (i,j) where user i has rated item j

we're going to take our models predicted ratings, compare them to the actual ratings, square the difference and then take the average of those squared differences